<a href="https://colab.research.google.com/github/Hind-Saleh-Alatawi/Domain_Specific_embeddings_with_deep_Model_classifier/blob/main/Final_word2vec_and_deep_model_all_the_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip uninstall tensorflow


Uninstalling tensorflow-1.15.2:
  Would remove:
    /tensorflow-1.15.2/python3.6/tensorflow-1.15.2.dist-info/*
    /tensorflow-1.15.2/python3.6/tensorflow/*
    /tensorflow-1.15.2/python3.6/tensorflow_core/*
Proceed (y/n)? 

In [ ]:
pip install tensorflow==1.14

In [ ]:
%tensorflow_version 1.x
import re
import numpy as np
from bs4 import BeautifulSoup
from sklearn import metrics
import pandas as pd
from keras.models import Sequential
from string import punctuation
from keras.layers import Dense, Embedding, Flatten,Dropout, Activation,LSTM,Bidirectional,CuDNNLSTM
from sklearn.linear_model import LogisticRegression
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
import os
import io
import numpy as npvocab
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import matplotlib.pyplot as plt
plt.style.use('ggplot')
xrange=range
#NAME ='CNN-word2vec-600Users-50and25Dense'
#filepath='Bbalanced_Dataset_20k_20k.csv'
#filepath='StormfrontdatasetBALANCED.csv'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'

from sklearn.model_selection import train_test_split
'''
'Pre-Waseem-EMNLP': '/content/drive/My Drive/hate speech/Datasets Hatespeech/Preprocessed /PreprocessedWaseem-EMNLP.csv',

                 'Pre-Waseem-NAACL': '/content/drive/My Drive/hate speech/Datasets Hatespeech/Preprocessed /PreprocessedWaseem-NAACL.csv',

                 'Pre-Davidson': '/content/drive/My Drive/hate speech/Datasets Hatespeech/Preprocessed /Preprocessedlabeled_data_Davidson_27000_ready.csv',
                 'stormfront': '/content/drive/My Drive/white _supremacist/Stormfrontdataset.csv',
                 'ourdataset': '/content/drive/My Drive/white _supremacist/Batch_text_Voting_labels_ready.csv',
                 'Pre_Combined_balanced':'/content/drive/My Drive/hate speech/Datasets Hatespeech/Preprocessed /Preprocessed LabeledDavidsonWaseemCombinedBalanced32512.csv',
                                  'SubsetCombinedBalancedWS': '/content/drive/My Drive/white _supremacist/BalancedSubsetStormfrontdataset.csv'
'Waseem-EMNLP': '/content/drive/My Drive/hate speech/Datasets Hatespeech/Final_Preprocessed_Hate_RemoveUserName/Final_Preprocessed_datasets_WUNWaseem-EMNLP.csv',
                 'Waseem-NAACL': '/content/drive/My Drive/hate speech/Datasets Hatespeech/Final_Preprocessed_Hate_RemoveUserName/Final_Preprocessed_datasets_WUNWaseem-NAACL.csv',
                 'Combined_balanced':'/content/drive/My Drive/hate speech/Datasets Hatespeech/Final_Preprocessed_Hate_RemoveUserName/Final_Preprocessed_datasets_WUNLabeledDavidsonWaseemCombinedBalanced32512.csv'
'''
filepath_dict = {
                 'PreprocessedWScombined balanced': '/content/drive/My Drive/white _supremacist/Preprocessed/Preprocessed WS datasetscombined balanced ws.csv',
                 'PreprocessedStormfrontdataset': '/content/drive/My Drive/white _supremacist/Preprocessed/Preprocessed WS datasetsStormfrontdataset.csv',
                 'PreprocessedTwitterdataset': '/content/drive/My Drive/white _supremacist/Preprocessed/Preprocessed WS datasetsBatch_text_Voting_labels_ready.csv'

                                 }
df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath,names=['text','label'],sep=',',skiprows=[0])#skiprows=[0] to skip header file text,label from each file
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)
df = pd.concat(df_list)
df.to_csv("df.csv")
print(df)



def clean_doc(doc):
    #for row in doc:
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    #tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

def makedocument(texts):
    # load all docs in a directory
    documents=list()
    for text in texts:
     tokens = clean_doc(text)
     #print(tokens)
     # add to list
     documents.append(tokens)
    return documents
all_outputs=[]
from sklearn.model_selection import train_test_split
for source in df['source'].unique():
    df_source = df[df['source'] == source]
    sentences = df_source['text'].values
    y = df_source['label'].values
    def max_length(lines):
        return max([len(s.split()) for s in lines])
    DatasetName=source
    print("Processing dataset...."+DatasetName)
    df_ChoosenDataset = df[df['source'] == DatasetName]
    Tweets = df_ChoosenDataset['text'].values
    labels = df_ChoosenDataset['label'].values
    number_of_labels = df_ChoosenDataset['label'].value_counts()
    print('number_of_labels is :\n',number_of_labels)
    #Tweets=makedocument(Tweets)
    Tweets=makedocument(Tweets)
    Tweets_train, Tweets_test, y_train, y_test = train_test_split(Tweets, labels,test_size=0.2,random_state=1000)# stratify=labels,
    print('Tweets_train:',Tweets_train[:5])
    #\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\num_words=10000,split=' '
    tokenizer = Tokenizer()
    allsentences=Tweets_train+Tweets_test
    tokenizer.fit_on_texts(allsentences)
    for word in ['kill', 'muslim', 'happy', 'black']:
        print('{}: {}'.format(word, tokenizer.word_index[word]))
    encoded_docs = tokenizer.texts_to_sequences(Tweets_train)
    X_test = tokenizer.texts_to_sequences(Tweets_test)
    vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
    maxlen = max([len(s.split()) for s in Tweets])
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    X_train = pad_sequences(encoded_docs, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
    '''
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print('Accuracy for data: {:.4f}'.format(score))
    '''
    #____________________________________________________________________
    ## I should fix .bin loading
    from gensim.models import Word2Vec, KeyedVectors
    t_model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,binary=True)
    #EmbeddingFile='/content/drive/My Drive/Models/All_Twitter_Papers_withHateWords_1M_300_CBOW2.txt'
    #raw_embedding = load_embedding(EmbeddingFile)
    #embedding_vectors = get_weight_matrix(t_model, tokenizer.word_index,embedding_dim)
    # create the embedding layer
    #print(embedding_vectors)
    # define model this is different frm original one
    #class_names = ["HATE","NONHATE"]
    #First, let’s have a quick look how many of the embedding vectors are nonzero:
    #nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_vectors, axis=1))
    #similarity_percentage=nonzero_elements / vocab_size
    #This means 95.1% of the vocabulary is covered by the pretrained model, which is a good coverage of our vocabulary. Let’s have a look at the performance when using the GlobalMaxPool1D layer:
    #print("{} of the vocabulary is covered by the pretrained model for {}".format(similarity_percentage,DatasetName))
    #to load my model I should use this statement and change each 300 to 100
    # t_model = Word2Vec.load('embedding_word2vec_Stormfronttextonly.model')
    embeddings_index = {}
    for line in Tweets:
        for word in line.split(" "):
            if word in t_model.wv:
              coefs = t_model.wv[word]
              embeddings_index[word] = coefs
            else:embeddings_index[word] = [np.zeros(300)]

    EMBEDDING_DIM =300
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

    for word, i in word_index.items():
        if i > num_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = np.array(embedding_vector)

    #
    nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
    similarity_percentage=nonzero_elements / vocab_size
    embedding_layer = Embedding(num_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=maxlen)#,trainable=True
    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(CuDNNLSTM(75)))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(1, activation='sigmoid'))# compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history=model.fit(X_train, y_train, epochs=10,batch_size=256,validation_data=(X_test, y_test))
    print(model.summary())
    # compile network
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#optimizer=rmsprop
    # fit the model
    #history=model.fit(X_train, y_train,  epochs=10, verbose=2,validation_data=(X_test, y_test))#
    loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Training Accuracy:",(accuracy),'Training Loss:',loss)
    loss,Accuracy=scores
    print("validation Accuracy:",Accuracy,' validation loss:',loss)
    #plot_history(history)
    #plt.show()
    # evaluate X_train
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print("Testing Accuracy:  {:.4f}".format(acc))
    y_predict=model.predict(X_test)
    print(classification_report(y_predict.round(), y_test))
    accuracy=accuracy_score(y_test, y_predict.round())
    print('DEEP MODEL Accuracy:',accuracy)
    precision=precision_score(y_test, y_predict.round())
    print('DEEP MODEL precision_score:', precision)
    recall=recall_score(y_test, y_predict.round())
    print('DEEP MODEL recall_score:',recall )
    f1=f1_score(y_test, y_predict.round())
    print('DEEP MODEL f1_score:',f1 )
    f0=f1_score(y_test, y_predict.round(),pos_label=0)
    print('DEEP MODEL f1_score of Zero:',f1 )
    fweighted=f1_score(y_test, y_predict.round(),average='weighted')
    print('DEEP MODEL f1_score of fweighted:',fweighted )
    fMacro=f1_score(y_test, y_predict.round(),average='macro')
    print('DEEP MODEL f1_score of fMacro:',fMacro )
    fMicro=f1_score(y_test, y_predict.round(),average='micro')
    print('DEEP MODEL f1_score of fMicro:',fMicro )
    ClassificationReport=classification_report(y_test, y_predict.round())
    print('DEEP MODEL ClassificationReport:',fMicro )
    fpr, tpr, thresholds = metrics.roc_curve(y_test,y_predict.round())
    AUC=metrics.auc(fpr, tpr)
    print('DEEP MODEL AUC: ' ,AUC)
    import seaborn as sns
    labels=["neutral","hate"]
    cm =confusion_matrix(y_test, y_predict.round())
    print("confusion matrix",cm)
    index = ['neutral','hate']
    columns = ['neutral','hate']
    cm_df = pd.DataFrame(cm,columns,index)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm_df, annot=True,cmap=plt.cm.Blues)
    plt.ylabel('True label')
    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    #plt.show()
    csv_file=source+"ConfusionMatrix"
    plt.savefig(csv_file.split('.csv')[0] + '.png')
    from datetime import datetime

    now = datetime.now()
    model_summary=model.summary()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    def get_model_summary(model):
        stream = io.StringIO()
        model.summary(print_fn=lambda x: stream.write(x + '\n'))
        summary_string = stream.getvalue()
        stream.close()
        return summary_string
    model_summary_string = get_model_summary(model)
    totals = [DatasetName,similarity_percentage, accuracy, precision, recall,f1,f0,fweighted,fMacro,fMicro,AUC,dt_string,model_summary_string,ClassificationReport]
    all_outputs.append(totals)
    mytest = "Google hate die"
    encoded_mytest = tokenizer.texts_to_sequences([mytest])
    padded_mytest = pad_sequences(encoded_mytest, maxlen=maxlen, padding='post')
    yhat = model.predict(padded_mytest)
    print('prediction of sentence',yhat)
result = pd.DataFrame(data=all_outputs, columns=['Dataset Name','similarity_percentage', 'accuracy', 'precision', 'recall','f1','f0','fweighted','fMacro','fMicro','AUC','today date Time','model summary','ClassificationReport'])
result.to_csv("FinalPretrainedDeepModelResult"+datetime.now().strftime("%d-%m-%Y_%I-%M-%S_%p")+".csv")
result.to_csv("/content/drive/My Drive/hate speech/Datasets Hatespeech/"+"FinalPretrainedDeepModelResult"+datetime.now().strftime("%d-%m-%Y_%I-%M-%S_%p")+".csv")
cm = confusion_matrix(y_test, y_predict.round())

print(cm)

#Now the normalize the diagonal entries

cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

print(cm)

#The diagonal entries are the accuracies of each class

cm.diagonal()

#array([1.        , 0.        , 0.66666667])